In [1]:
import os
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
from rmgpy.species import Species
from rmgpy.molecule import Molecule

In [10]:
def addThermoData(thermoData1, thermoData2):
    """
    Add the thermodynamic data `thermoData2` to the data `thermoData1`,
    and return `thermoData1`.
    """
    if len(thermoData1.Tdata.value_si) != len(thermoData2.Tdata.value_si) or any([T1 != T2 for T1, T2 in zip(thermoData1.Tdata.value_si, thermoData2.Tdata.value_si)]):
        raise Exception('Cannot add these ThermoData objects due to their having different temperature points.')

    for i in range(thermoData1.Tdata.value_si.shape[0]):
        thermoData1.Cpdata.value_si[i] += thermoData2.Cpdata.value_si[i]
    thermoData1.H298.value_si += thermoData2.H298.value_si
    thermoData1.S298.value_si += thermoData2.S298.value_si

    #if thermoData1.comment:
    #    thermoData1.comment += ' + {0}'.format(thermoData2.comment)
    #else:
    #    thermoData1.comment = 'Thermo group additivity estimation: ' + thermoData2.comment

    return thermoData1

def removeThermoData(thermoData1, thermoData2):
    """
    Remove the thermodynamic data `thermoData2` from the data `thermoData1`,
    and return `thermoData1`.
    """
    if len(thermoData1.Tdata.value_si) != len(thermoData2.Tdata.value_si) or any([T1 != T2 for T1, T2 in zip(thermoData1.Tdata.value_si, thermoData2.Tdata.value_si)]):
        raise Exception('Cannot add these ThermoData objects due to their having different temperature points.')

    for i in range(thermoData1.Tdata.value_si.shape[0]):
        thermoData1.Cpdata.value_si[i] -= thermoData2.Cpdata.value_si[i]
    thermoData1.H298.value_si -= thermoData2.H298.value_si
    thermoData1.S298.value_si -= thermoData2.S298.value_si

    #if thermoData1.comment:
    #    thermoData1.comment += ' + {0}'.format(thermoData2.comment)
    #else:
    #    thermoData1.comment = 'Thermo group additivity estimation: ' + thermoData2.comment

    return thermoData1

In [3]:
libraryName = 'C10H11'
database = RMGDatabase()
database.load(settings['database.directory'], thermoLibraries = [libraryName], kineticsFamilies='none', kineticsDepositories='none', reactionLibraries=[])

thermoDatabase = database.thermo
thermoLibrary = database.thermo.libraries[libraryName]

In [4]:
for label, entry in thermoLibrary.entries.iteritems():
    molecule = entry.item
    thermoData = entry.data
    #print molecule
    #print thermoData

In [5]:
m = Molecule().fromInChI('InChI=1S/C9H8/c1-2-5-9-7-3-6-8(9)4-1/h1-6H,7H2')
s = Species(molecule=[m])
s.generateResonanceIsomers()

In [6]:
estimatedThermo = thermoDatabase.getThermoDataFromGroups(s)
print estimatedThermo

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([119.118,159.661,197.401,230.287,278.403,306.813,353.171],'J/(mol*K)'), H298=(157.235,'kJ/mol'), S298=(334.971,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(407.409,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cb-Cs) + other(R) + group(Cb-(Cds-Cds)) + other(R) + group(Cs-(Cds-Cds)CbHH) + other(R) + group(Cds-CdsCsH) + other(R) + group(Cds-CdsCbH) + other(R) + group(Cb-H) + other(R) + group(Cb-H) + other(R) + group(Cb-H) + other(R) + group(Cb-H) + other(R) + polycyclic(indene)""")


In [7]:
polycyclicGroup = thermoDatabase.groups['polycyclic'].entries['indene']
print polycyclicGroup.data


ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([-3.88,-4.29,-3.89,-2.96,-1.83,-2.2,-1.31],'cal/(mol*K)'), H298=(2.1,'kcal/mol'), S298=(33.08,'cal/(mol*K)'), comment="""polycyclic(indene)""")


In [13]:
withoutPolycyclicGroupThermo = removeThermoData(estimatedThermo, polycyclicGroup.data)
print withoutPolycyclicGroupThermo

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([151.586,195.56,229.953,255.057,293.717,325.222,364.134],'J/(mol*K)'), H298=(139.662,'kJ/mol'), S298=(58.1576,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(407.409,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cb-Cs) + other(R) + group(Cb-(Cds-Cds)) + other(R) + group(Cs-(Cds-Cds)CbHH) + other(R) + group(Cds-CdsCsH) + other(R) + group(Cds-CdsCbH) + other(R) + group(Cb-H) + other(R) + group(Cb-H) + other(R) + group(Cb-H) + other(R) + group(Cb-H) + other(R) + polycyclic(indene)""")


In [16]:
libraryThermoData = thermoLibrary.entries['INDENE'].data

In [17]:
newPolycyclicGroupThermo = removeThermoData(libraryThermoData, withoutPolycyclicGroupThermo)

In [20]:
from rmgpy.cantherm.output import prettify
print prettify(repr(newPolycyclicGroupThermo))
print prettify(repr(polycyclicGroup.data))

ThermoData(
    Tdata = ([300, 400, 500, 600, 800, 1000, 1500], 'K'),
    Cpdata = ([-6.95, -6.96, -6.31, -5.18, -4.03, -4.43, -3.35], 'cal/(mol*K)'),
    H298 = (7.11, 'kcal/mol'),
    S298 = (65.92, 'cal/(mol*K)'),
)
ThermoData(
    Tdata = ([300, 400, 500, 600, 800, 1000, 1500], 'K'),
    Cpdata = ([-3.88, -4.29, -3.89, -2.96, -1.83, -2.2, -1.31], 'cal/(mol*K)'),
    H298 = (2.1, 'kcal/mol'),
    S298 = (33.08, 'cal/(mol*K)'),
    comment = 'polycyclic(indene)',
)


In [22]:
print repr(polycyclicGroup)

<Entry index=11 label="indene">


In [ ]:
# At this point, overwrite just the thermo data of the original entry for the thermoGroup, also indicate 
# how it was fitted in the description
polycyclicGroup.data = newPolycyclicGroupThermo
polycyclicGroup.shortDesc = "Fitted from thermo library values"
polycyclicGroup.longDesc = "Fitted from {0} species from {1} library.".format('blah','blahablakjs')

# Then save the entry to disk by appending or rewriting the thermo polycyclic.py file.